In [ ]:
import random

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas
import pandas as pd
import seaborn as sns
import yaml
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.datasets import make_circles, make_blobs, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree


SEED = 314159
TRAIN_TEST_SPLIT = 0.80

data_path = "D:\data\machine_learning"

Рассмотрим набор данных о качестве вина на основе различных химических показателей. Есть всего 6 значений качества, поэтому задачу проще всего решать классификацией.

In [ ]:
df = pd.read_csv(data_path+'/'+"winequality-red.csv")

In [ ]:
df.describe()

## Несбалансированные данные
Несбалансированные данные - это одна из наиболее распространенных проблем, с которой сталкиваются в машинном обучении. Они возникают, когда классы, которые мы пытаемся предсказать, представлены неравномерно. Другими словами, один класс имеет гораздо меньше примеров, чем другой, что создает дисбаланс в распределении данных.
Деревья решений очень уязвимы перед дисбалансом классов, так как используют вероятности в критериях разбиения.

In [ ]:
X = df.drop('quality', axis=1)
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))

### Метрики
Так как данные несбалансированные, уместно ли использовать встроенное значение score для оценки модели? На самом деле, и да и нет, так как score возвращает усредненную точность по классам. Однако еще более информатиными будут F1-мера и другие более подробные оценки.
Формула Precision:
\begin{equation}
\text{Precision} = \frac{\text{TP}}{ \text{TP} + \text{FP} }
\end{equation}
Precision наиболее полезна, если цена FP велика. Примеры такой задачи - детекция спама. В случае, если полезное письмо попадет в спам, юзер может потерять важную информацию, что нежелательно.
Формула Recall:
\begin{equation}
\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}
\end{equation}
Recall наиболее полезна, если цена FN велика. Она поможет, если нам необходимо найти всех больных пациентов, определить фрод и т.д.
Вопрос: что отражают две эти метрики?


Формула F1-метрики:
\begin{equation}
\text{F1} = \frac{2 * \text{TP}}{2 * \text{TP} + \text{FP} + \text{FN}}
\end{equation}
F1-метрика может быть выведена из precision и recall и позволяет найти баланс между ними. Вопрос - как она выводится?

Существует и болеее общая оценка - $F_\beta$. Она позволяет получить взвешенное среднее между precision и recall. Ее формулу дадим на занятии. Наиболее популярные параметры - 2 и 0.5.
Тут тоже есть особенности. Метрики бывают нескольких видов, в зависимости от того, как учитываются разные классы:
- микро - метрики считаются глобально сразу по всем классам.
- макро - вначале считаются для каждого класса, потом берется среднее
- взвешенные по классам - вначале считаются для каждого класса, потом берется среднее, взвешенное по поддержке

Вопрос: какой способ вычисления лучше всего выбрать при дисбалансе классов? Покажите на примере.

In [ ]:
from sklearn.metrics import f1_score

print("Train f1-score is: ", f1_score(y_train, model.predict(X_train), average="micro"))
print("Test f1-score is: ", f1_score(y_test, model.predict(X_test), average="micro"))

In [ ]:
from sklearn.metrics import f1_score

print("Train f1-score is: ", f1_score(y_train, model.predict(X_train), average="macro"))
print("Test f1-score is: ", f1_score(y_test, model.predict(X_test), average="macro"))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(model, X_test, y_test)
plt.show()

Давайте руками рассмотрим, как оценивается точность и почему нам сложно получить адекватную оценку при сильном дисбалансе.

In [ ]:
metrics_df = pd.DataFrame(columns=['label', 'TP', 'TN', 'FP', 'FN', 'support', 'accuracy', 'precision', 'recall'])

In [ ]:
from sklearn.utils import compute_class_weight
classes = np.unique(y_train)
classes.sort()
print(classes)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
for class_id in classes:
    class_stats = {}
    # your code
    metrics_df = pd.concat([metrics_df, pd.DataFrame(class_stats, index=[0])], ignore_index=True)

## Imbalanced learning

Как заставить модель учиться так, чтобы учитывать все классы? Самое простое - задать вес класса.

In [ ]:
class_weights = # your code
model = DecisionTreeClassifier(class_weight={classes[i]: w for i, w in enumerate(class_weights)})
model.fit(X_train, y_train)
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))

Выведем, как изменились предсказания модели.

In [ ]:
# create new dataframe with metrics

Также посмотрим, насколько чувствительна к дисбалансу другая модель - LogisticRegression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:
logreg = # make pipeline of Scaler and Logreg
logreg.fit(X_train, y_train)
print("Naive train score is: ", logreg.score(X_train, y_train))
print("Naive test score is: ", logreg.score(X_test, y_test))
print("Classes", logreg.classes_)

In [ ]:
logreg = # make pipeline of Scaler and Logreg with class weights
logreg.fit(X_train, y_train)
print("Balanced train score is: ", logreg.score(X_train, y_train))
print("Balanced test score is: ", logreg.score(X_test, y_test))

То же самое можно сделать с использованием итеративного линейного классификатора. Используем для этого SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier
logreg = # make pipeline of Scaler and SGDClassifier without class weights
logreg.fit(X_train, y_train)
print("Naive train score is: ", logreg.score(X_train, y_train))
print("Naive test score is: ", logreg.score(X_test, y_test))
print("Naive test f1-score is: ", f1_score(y_test, y_pred=logreg.predict(X_test), average="macro"))

In [ ]:
logreg = # make pipeline of Scaler and SGDClassifier with class weights
logreg.fit(X_train, y_train)
print("Balanced train score is: ", logreg.score(X_train, y_train))
print("Balanced test score is: ", logreg.score(X_test, y_test))
print("Balanced test f1-score is: ", f1_score(y_test, y_pred=logreg.predict(X_test), average="macro"))

Как видно, балансировка не всегда одинаково полезна. Однако мы можем попробовать воспользоваться дополнительными методами для обучения нашего дерева. Это будет андер- и оверсемплинг (балансировка датасета). Самое простое - взять только небольшую, но сбалансированную часть обучающих данных для непосредственного обучения. Однако, для нашего случая это плохо подходит, так как данных довольно мало.

Еще раз вспомним, какие предсказания давало дерево решений для нашего набора

In [ ]:
model = DecisionTreeClassifier(max_depth=5)
model.fit(X_train, y_train)
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))
print("Test f1-score is: ", f1_score(y_test, y_pred=model.predict(X_test), average="macro"))

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=SEED, ) # choose your parameters
X_res, y_res = ros.fit_resample(X_train, y_train)
y_res.value_counts()

model = DecisionTreeClassifier(max_depth=5)
model.fit(X_res, y_res)
print("Train score is: ", model.score(X_res, y_res))
print("Test score is: ", model.score(X_test, y_test))
print("Test f1-score is: ", f1_score(y_test, y_pred=model.predict(X_test), average="macro"))

### SMOTE (Synthetic Minority Oversampling Technique):
SMOTE позволяет расширить выборку, построив синтетические примеры на основе уже существующих. Это еще один вариант решения «проблем классового дисбаланса».

Общая схема SMOTE:
1) Для создания нового семпла используют пару признаков «соседних» примеров a  и b из миноритарного класса. Их находят, используя алгоритм ближайшего соседа KNN. В данном случае необходимо и достаточно для семпла a получить набор из k соседей, из которого в дальнейшем будет случайно выбран семпл b.
2) Для создания нового семпла находят разность $d=X_b–X_a$, где $X_a,X_b$ – векторы признаков «соседних» примеров a  и b из миноритарного класса.
3) Далее из $d$ путем его умножения на случайное число в интервале (0,1)  получают $d'$.
4) Вектор признаков нового примера вычисляется путем сложения Xa и $d'$.

Вопрос: сколько соседей выбирать? И сколько соседей стоит использовать для каждой узловой точки?


In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=SEED) # choose your parameters
X_res, y_res = smote.fit_resample(X_train, y_train)

In [ ]:
y_res.value_counts()

In [ ]:
smote = SMOTE(random_state=SEED) # choose your parameters
X_res, y_res = smote.fit_resample(X_train, y_train)
y_res.value_counts()

In [ ]:
model = DecisionTreeClassifier(max_depth=5)
model.fit(X_res, y_res)
print("Train score is: ", model.score(X_res, y_res))
print("Test score is: ", model.score(X_test, y_test))
print("Test f1-score is: ", f1_score(y_test, y_pred=model.predict(X_test), average="macro"))

In [ ]:
smote = SMOTE(random_state=SEED) # choose your parameters
X_res, y_res = smote.fit_resample(X_train, y_train)
y_res.value_counts()

In [ ]:
model = DecisionTreeClassifier(max_depth=5)
model.fit(X_res, y_res)
print("Train score is: ", model.score(X_res, y_res))
print("Test score is: ", model.score(X_test, y_test))
print("Test f1-score is: ", f1_score(y_test, y_pred=model.predict(X_test), average="macro"))

Также мы можем получить более подробный отчет по точности модели.

In [ ]:
from imblearn.metrics import classification_report_imbalanced

In [ ]:
print(classification_report_imbalanced(y_test, model.predict(X_test)))

Рассмотрим также поведение разных алгоритмов на синтетических данных.

In [ ]:
X_gen, y_gen = make_classification(n_samples=5000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=3,
                           n_clusters_per_class=1,
                           weights=[0.01, 0.05, 0.94],
                           class_sep=0.8, random_state=SEED)
X_train, X_test, y_train, y_test = # train test split

ros = RandomOverSampler(random_state=SEED)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)



In [ ]:
# Choosing the first 2 columns for the plot

def get_decision_boundary(X_train, y_train, features, model="logistic"):
    # Creating and fitting the tree classifier
    if model == "logistic":
        classifier = # your code
    else:
        classifier = # your code
    # Plotting the tree boundaries
    disp = # your code

    # Plotting the data points
    # your code

    plt.title(f"Decision surface for {classifier.__class__.__name__} trained on {features[0]} and {features[1]}")
    plt.show()


In [ ]:
get_decision_boundary(X_train, y_train, features = ['x1', 'x2' ])

In [ ]:
get_decision_boundary(X_resampled, y_resampled, features = ['x1', 'x2' ])

In [ ]:
smote = SMOTE(random_state=SEED, k_neighbors=4)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
get_decision_boundary(X_resampled, y_resampled, features = ['x1', 'x2' ])

Введем еще один метод оверсемплинга - AdaSYN (Adaptive Synthetic). Этот алгоритм, в отличие от ресэмплинга и SMOTE позволяет сгенерировать более "сложные" примеры.
Этот алгоритм работает следующим образом:
1) Рассчитывается отношение миноритарного и мажоритарного класса: $d = m_s / m_l $
2) Определяется, сколько элементов необходимо нагенерировать: $G = (m_l - m_s) * \beta $. $\beta $ - параметр балансировки
3) Для каждого семпла из миноритарного класса находятся k соседей. После чего ему присваивается вес $r_i = num_major_neighbours / k $.
4) $r_i$ нормализуются так, чтобы их сумма равнялсь 1: $ {r_i}' =  \frac {r_i} {\sum{r_j}$
5) Для каждого семпла определяется число сгененрированных примеров.  $G_i = G * {r_i}' $. Вопрос: в каких областях будет генерироваться больше примеров?
6) Далее для каждого примера и его соседей генерируется синтетический пример, как в SMOTE. Также может быть использована добавка белого шума.

Вопрос: какие минусы у SMOTE и AdaSYN по отношению друг к другу?

In [ ]:
from imblearn.over_sampling import ADASYN
smote = ADASYN(random_state=SEED) # your parameters
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
get_decision_boundary(X_resampled, y_resampled, features = ['x1', 'x2' ])

На графике сразу видно, когда AdaSYN  может помешать хорошей генерации. Как видно, так как AdaSYN концентрирует сгенерированные примеры у границ классов, тогда как SMOTE может "соединять" и примеры в центральной части класса, то он может быть более удачным методом. К тому же, сущестуют различные вариации SNOTE, которые могут давать хороший рещультат в такой постановке.

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
smote = BorderlineSMOTE(random_state=SEED) # your parameters
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
get_decision_boundary(X_resampled, y_resampled, features = ['x1', 'x2' ])

Вопрос: какие можно еще придумать более "умные" варианты оверсемплинга на основе CMOTE?

## Уменьшение выборки (андерсемплинг)

### Prototype generation
Методы Prototype generation (создания прототипов) позволяют сократить количество объектов в целевых классах, но остальные объектов генерируются, а не выбираются, из исходного набора.
ClusterCentroids использует для этого KMeans.

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
sampler = ClusterCentroids(random_state=SEED, estimator=# your parameters)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
get_decision_boundary(X_resampled, y_resampled, features = ['x1', 'x2' ])

In [ ]:
from imblearn.under_sampling import ClusterCentroids
sampler = ClusterCentroids(random_state=SEED, estimator=# your parameters)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
get_decision_boundary(X_resampled, y_resampled, features = ['x1', 'x2' ])

### Prototype selection
Алгоритмы Prototype selection (выбора прототипов) можно разделить на две группы: (i) методы контролируемой недостаточной выборки и (ii) методы очистки недостаточной выборки.

Методы контролируемой недостаточной выборки сокращают количество примеров в мажоритарных классах до произвольного размера, указанного пользователем. Обычно они сводят количество наблюдений к размеру миноритарного класса(ов).
К ним относятся: Random UnderSampling, NearMiss
Напротив, методы очистки недостаточной выборки «очищают» пространство признаков, удаляя либо «зашумленные», либо «слишком легко классифицируемые» наблюдения, в зависимости от метода. Окончательное количество наблюдений в каждом классе зависит от метода очистки и не может быть указано пользователем.
К ним относится, например, метод связей Томека.


Наиболее простой метод - RandomUnderSampler - просто выкидывает семплы случайным образом.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
get_decision_boundary(X_resampled, y_resampled, features = ['x1', 'x2' ])

Попробуйте реализовать такой семплер вручную:

In [ ]:
# your code

NearMiss добавляет некоторые эвристики для выбора семплов. Так, он может выбирать для удаления примеры, ближайшие к миноритарном классу, примеры, среднее расстояние у которых до наиболее дальних примеров из миноритарного класса наименьшее.

In [ ]:
# you can add some figures with nearmiss undersampling

Связь Томека существует, когда два примера из разных классов являются ближайшими соседями друг к другу. Алгоритм связей Томека детектирует такие связи и удаляет пример из мажоритарного класса. Основная идея заключается в том, что связи Томека зашумлены или наблюдения трудно классифицировать, и поэтому они не помогут алгоритму найти подходящую границу разделения.


In [ ]:
# youк сode

Edited nearest neighbours - для примеров из миноритарного класса находит k ближайших соседей и удаляет все семплы из мажоритарных классов, попавшие в эту окрестность.

In [ ]:
# your code